# Run Vcontact2 with the prophage genomes
***

Generate the input files 

In [ ]:
# Create a tmp with all the candidates :

import os
import pandas as pd
import random
import sys
import subprocess
from tqdm import tqdm


path_phageboost_pred="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_prediction"
path_fasta="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022"
path_fastani = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022_out"
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_ktype="/home/conchae/prediction_depolymerase_tropism"


df_representant = pd.read_csv(f"{path_fastani}/clusters_99_80.extra_clean.2004.v2.tsv", sep = "\t", header = 0)
df_representant = df_representant.drop_duplicates(subset = ["prophage_id"], keep = "first")

dico_prophage = {}

for _,row in df_representant.iterrows() :
    strain = row["prophage"].split("__")[0]
    prophage = row["prophage"].split(".fasta")[0].split("__")[-1]
    seq_id = row["prophage_id"]
    a = {"strain" : strain, "prophage" : prophage, "prophage_n" : seq_id}
    dico_prophage[row["prophage"]] = a
        
for phage in tqdm(dico_prophage):       
    strain = dico_prophage[phage]["strain"]
    prophage_id = dico_prophage[phage]["prophage"]
    prophage = phage.split(".fasta")[0]
    genecall = pd.read_csv(f"{path_phageboost_pred}/{strain}/genecalls_{strain.split('.')[0]}.gff3", sep="\t")
    for file in os.listdir(f"{path_phageboost_pred}/{strain}"):
        if file[0:6]=="phages":
            resume= pd.read_csv(f"{path_phageboost_pred}/{strain}/{file}", skiprows=1, sep="\t")
            break
    attributes = resume["attributes"].to_list()
    for index_att, attri in enumerate(attributes):
        if attri.split("phage_id=")[1] == prophage_id :
            attribute_line = attri
            break
    frag_id = resume[resume["attributes"] == attribute_line]["#seqid"].values[0]
    start_genome ,stop_genome =resume[resume["attributes"] == attribute_line]["start"].values[0], resume[resume["attributes"] == attribute_line]["end"].values[0]
    print(frag_id ,start_genome, stop_genome)
    #break
    genecall_frag = genecall[genecall["contig"] == frag_id]
    genecall_frag_dict = genecall_frag.to_dict('records')
    with open(f"{path_decipher}/{strain}/{prophage_id}.multi.faa", "w") as outfile_faa :
        with open(f"{path_decipher}/{strain}/{prophage_id}.multi.ffn", "w") as outfile_ffn :
            for line in genecall_frag_dict : 
                if line["start"] in range(start_genome, stop_genome) :
                    nt_seq, aa_seq, prot_id =line["DNAseq"] , line["AAseq"],  line["id"]
                    print(len(aa_seq), "Protein_id : ", prot_id)
                    if len(aa_seq) > 10 :
                        outfile_faa.write(f">{strain}__{prophage_id}__{prot_id}\n{aa_seq}\n")
                        outfile_ffn.write(f">{strain}__{prophage_id}__{prot_id}\n{nt_seq}\n")
                        print(f">{strain}__{prophage_id}__{prot_id}\n{aa_seq}\n", f">{strain}__{prophage_id}__{prot_id}\n{nt_seq}\n")
                
    

In [ ]:
#!/bin/bash
#BATCH --job-name=write_multi
#SBATCH --qos=short 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=4
#SBATCH --mem=5gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=write_multi%j.log 

module restore la_base
conda activate python_311
                
python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/script_file/make_multifasta.py

In [ ]:
# Step one : generating the fasta file with the phages proteins and the gene_to_genome file
import os
import pandas as pd
from tqdm import tqdm
from Bio import SeqIO

path_phageboost = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_phylogeny = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny"
path_fastani = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022_out"

df_representant = pd.read_csv(f"{path_fastani}/clusters_99_80.extra_clean.2004.v2.tsv", sep = "\t", header = 0)
df_representant = df_representant.drop_duplicates(subset = ["prophage_id"], keep = "first")

dico_prophage = {}

for _,row in df_representant.iterrows() :
    strain = row["prophage"].split("__")[0]
    prophage = row["prophage"].split(".fasta")[0].split("__")[-1]
    seq_id = row["prophage_id"]
    a = {"strain" : strain, "prophage" : prophage, "prophage_n" : seq_id}
    dico_prophage[row["prophage"]] = a
    
with open(f"{path_phylogeny}/phageboost_70_Vcontact2.0611.fasta","w") as outfile1 :
    with open(f"{path_phylogeny}/phageboost_70_gene_to_genome.0611.csv","w") as outfile2:
        outfile2.write("protein_id,contig_id,keywords\n")
        for prophage in tqdm(dico_prophage) :
            path_multi = f"{path_phageboost}/{dico_prophage[prophage]['strain']}/{dico_prophage[prophage]['prophage']}.multi.faa"
            fasta_file = SeqIO.parse(path_multi , "fasta")
            for index, record in enumerate(fasta_file) :
                prot_id = f"{prophage.split('.fasta')[0]}__{index}"
                outfile1.write(f">{prot_id}\n{str(record.seq)}\n")
                outfile2.write(f"{prot_id},{prophage.split('.fasta')[0]},None_provided\n")
                


Merge the previously created input files with the Millard data

In [ ]:
import os

path_phageboost="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70"
path_phylogeny="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny"
path_77="/home/conchae/77_strains_phage_project"
path_millard = "/home/conchae/databases/Millard_jan_2023"

gene2genome_m = "\n".join(open(f"{path_millard}/5Jan2023_vConTACT2_gene_to_genome.csv").read().split("\n")[1:])
gene_seq_m = open(f"{path_millard}/5Jan2023_vConTACT2_proteins.faa").read()

PPT_gene2genome = open(f"{path_phylogeny}/phageboost_70_gene_to_genome.0611.csv").read()
PPT_gene_seq = open(f"{path_phylogeny}/phageboost_70_Vcontact2.0611.fasta").read()

with open(f"{path_phylogeny}/phageboost_70_Vcontact2.Millard.0611.fasta","w") as outfile :
    with open(f"{path_phylogeny}/phageboost_70_gene_to_genome.Millard.0611.csv","w") as outfile2:
        outfile.write(f"{PPT_gene_seq}\n{gene_seq_m}")
        outfile2.write(f"{PPT_gene2genome}\n{gene2genome_m}")
        
        

In [ ]:
# Step two : The actual command line :

#!/bin/bash
#BATCH --job-name=vcontact2_phageboost
#SBATCH --qos=medium 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=40
#SBATCH --mem=200gb 
#SBATCH --time=2-00:00:00 
#SBATCH --output=vcontact2_phageboost%j.log 

module restore la_base
source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate vcontact2


vcontact --rel-mode 'Diamond' \
--raw-proteins /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/phageboost_70_Vcontact2.Millard.0611.fasta \
--proteins-fp /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/phageboost_70_gene_to_genome.Millard.0611.csv \
--output-dir  /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/Vcontact2_PPTmillard_output_07112023 \
--db 'None'  --pcs-mode MCL --vcs-mode ClusterONE --c1-bin /home/conchae/software/cluster_one-1.0.jar -t 40


# ***
vcontact --rel-mode 'Diamond' \
--contigs /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/Vcontact2_PPTmillard_output_07112023/vConTACT_contigs.csv \
--pcs /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/Vcontact2_PPTmillard_output_07112023/vConTACT_pcs.csv \
--pc-profiles /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/Vcontact2_PPTmillard_output_07112023/vConTACT_profiles.csv \
--raw-proteins /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/phageboost_70_Vcontact2.Millard.0611.fasta \
--proteins-fp /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/phageboost_70_gene_to_genome.Millard.0611.csv \
--output-dir  /home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/Vcontact2_PPTmillard_output_07112023_try2 \
--db 'None'  --pcs-mode MCL --vcs-mode ClusterONE --c1-bin /home/conchae/software/cluster_one-1.0.jar -t 40




In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_phylogeny/Vcontact2_PPTmillard_output_07112023 \
/media/concha-eloko/Linux/PPT_clean/  

***
# Infer the phylogeny :


In [23]:
import pandas as pd
import os
from tqdm import tqdm

path_df = "/media/concha-eloko/Linux/PPT_clean/Vcontact2_PPTmillard_output_07112023"
path_work = "/media/concha-eloko/Linux/PPT_clean"

    # Open the DF
DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df.tsv", sep = "\t" ,  header = 0)
df_vcontact = pd.read_csv(f"{path_df}/viral_cluster_overview.csv", sep = ",", header = 0, index_col = 0)


In [20]:
DF_info

,Phage,Protein_name,KL_type_LCA,Infected_ancestor,index,Dataset,seq,domain_seq,1,2,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,prophage_id
0,GCF_016651625.1__phage29,GCF_016651625.1__phage29__142,KL43,GCF_016651625.1,ppt__1,ppt,MSVPNQTPYNIYTANGLTTVFTYEFYIISASDLRVSINGDVVTSGY...,KDFVNINDYWFPTDGDDFYPALNKALSVSPHVLIPPGKHYLKSTVS...,-0.018416,0.022387,...,0.004437,0.087907,0.015800,0.025778,0.065790,0.034045,-0.070899,0.016068,0.065339,prophage_12186
1,GCF_016651625.1__phage29,GCF_016651625.1__phage29__150,KL43,GCF_016651625.1,anubis__0,anubis,MRANLIKTNFTAGEISPRLMGRVDIARYANGAKIIENAVCVVQGGV...,QAASPGAWTREDTVWTEEFGYPGAVTLYQQRLVLAGSPQYPQTIWW...,0.036016,0.005938,...,-0.037612,0.008772,0.010556,-0.049738,-0.012549,0.092624,-0.136602,-0.191378,0.135658,prophage_12186
2,GCF_016651625.1__phage12,GCF_016651625.1__phage12__59,KL43,GCF_016651625.1,ppt__4,ppt,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,NTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSDKYK...,0.026004,0.024372,...,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089,prophage_924
3,GCF_019928025.1__phage0,GCF_019928025.1__phage0__10,KL43,n1471,ppt__4,ppt,MSISKRNFLKAVSCAYFFYSFKALTKVNQPIEDYISTKDKNTWPSK...,NTWPSKVHRVEEFYTSTDRDYSDAILRGINYCSLNNCVLFFSDKYK...,0.026004,0.024372,...,-0.026018,0.018206,0.036751,-0.032549,0.064112,0.061520,-0.024423,-0.027998,0.028089,prophage_2929
4,GCF_004313505.1__phage4,GCF_004313505.1__phage4__113,KL14,GCF_004313505.1,anubis__5,anubis,MSEYDTGNPVPSASMPDAWDNMQSIDKFVNSSDETITTRTGQQLDT...,KAIFDAWLDFGIDWNGNESISLQLQTAVNYVSKLPYGGEIVLRPGV...,-0.023648,0.052674,...,-0.025991,0.068538,-0.051192,0.026481,0.069100,0.017813,-0.103797,0.018961,0.117058,prophage_11091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20134,GCF_002248635.1__phage4,GCF_002248635.1__phage4__44,KL102,n320,anubis_return__4264,anubis_return,MVSLKGMGSTFRDCTALISLPSGLLDGCINLTSLTLTFSGCTSLAL...,MVSLKGMGSTFRDCTALISLPSGLLDGCINLTSLTLTFSGCTSLAL...,-0.000585,-0.087093,...,0.036749,0.048489,0.020484,0.023950,-0.048109,0.134457,-0.101326,0.088485,0.037368,prophage_3054
20135,GCF_001905235.1__phage21,GCF_001905235.1__phage21__0,KL107,n35403540,anubis_return__4272,anubis_return,MLKHSLAIATCLAFSSSVMGNEANLLYTNTMQFPYKHNADGYMVFD...,VMGNEANLLYTNTMQFPYKHNADGYMVFDIHGKLVVPPEGHFDTLN...,0.076721,0.027635,...,0.042391,-0.004292,-0.004047,-0.011631,-0.026469,0.070159,-0.077212,-0.077950,-0.034630,prophage_313
20136,GCF_004312845.1__phage3,GCF_004312845.1__phage3__38,KL9,GCF_004312845.1,anubis_return__4275,anubis_return,MAILITGKSMTRLPESSSWEEEIELITRSERVAGGLDGPANRPLKS...,DAVIRRDLASDKGTSGVGKLGDKPLVAISYYKSKGQSDQDAVQAAF...,0.032196,0.048856,...,-0.016331,0.084711,0.056063,0.001793,0.073958,0.090169,-0.060105,0.023726,0.086452,prophage_12656
20137,GCF_900172635.1__phage2,GCF_900172635.1__phage2__1608,KL124,GCF_900172635.1,anubis_return__4287,anubis_return,MADLSISVISDQASESNQAGWWHPLDSFQGVEYYGLCKEYGTAGYH...,MADLSISVISDQASESNQAGWWHPLDSFQGVEYYGLCKEYGTAGYH...,-0.011089,-0.005328,...,0.034656,0.046130,0.012586,-0.021702,-0.023386,0.105700,-0.099147,-0.057367,0.091427,prophage_12780


In [10]:
df_vcontact

,VC,Size,Internal Weight,External Weight,Quality,P-value,Min Dist,Max Dist,Total Dist,Below Thres,Taxon Prediction Score,Avg Dist,Genera,Families,Orders,Members
0,VC_0_0,3,76.312457,0.000000,1.000000,1.000000e+00,1.000000,2.236068,3,3,1.000000,1.745356,0,0,0,"AB002632,KC357596,OP297622"
1,VC_1000_0,67,464895.630868,54594.076864,0.894908,0.000000e+00,0.000000,5.477226,2211,2211,1.000000,2.185786,0,0,0,"DQ398042,EU568876,FJ973624,GQ303261,JN412593,J..."
2,VC_1001_0,9,5315.878829,54472.603233,0.088911,5.886694e-24,3.000000,6.855655,36,36,1.000000,5.605551,0,0,0,"KJ410133,KR080197,KR080198,KR080202,MF141540,M..."
3,VC_1002_0,2,300.000000,4445.413917,0.063219,9.801281e-02,5.656854,5.656854,1,1,1.000000,5.656854,0,0,0,"MW584160,MW584202"
4,VC_1002_1,9,10800.000000,18388.248155,0.370012,3.126995e-20,0.000000,7.071068,36,36,1.000000,3.559221,0,0,0,"JN699628,KC900379,KT591491,MH697587,MK937612,M..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4093,VC_998_0,3,877.927279,951.053080,0.480009,3.826368e-03,3.000000,5.385165,3,3,1.000000,4.428048,0,0,0,"MN988507,MN988510,MN988527"
4094,VC_999_0,3,37.628323,19.699223,0.656374,4.848485e-02,6.928203,7.211103,3,3,1.000000,7.070125,0,0,0,"MW202785,MW202796,OK172329"
4095,VC_99_0,18,2354.511067,31197.670959,0.070175,2.505413e-08,2.236068,7.745967,153,153,1.000000,4.903771,0,0,0,"GCF_000786375.2__phage25,GCF_000812525.1__phag..."
4096,VC_9_0,3,362.655675,7090.130945,0.048660,2.931654e-03,7.000000,7.280110,3,3,1.000000,7.163737,0,0,0,"KX827371,MK075002,OM373558"


In [51]:
df_vcontact[df_vcontact["VC"] == 'VC_1097_0']["Members"].tolist()

['GCF_020117185.1__phage1,GCF_020121365.1__phage8']

In [30]:
path_millard_info = "/media/concha-eloko/Linux/Databases/Millard_jan_2023"

    # Dataframe with Vcontact information
df_family = pd.read_csv(f"{path_millard_info}/5Jan2023_vConTACT2_family_annotations.tsv", sep="\t")
df_sub_fam = pd.read_csv(f"{path_millard_info}/5Jan2023_vConTACT2_subfamily_annotations.tsv", sep="\t").rename(columns={'Sub-family': 'Sub_family'})
df_genus = pd.read_csv(f"{path_millard_info}/5Jan2023_vConTACT2_genus_annotations.tsv", sep="\t")


taxonomy = {}

for _, row in tqdm(df_vcontact.iterrows()) :
    for member in row["Members"].split(",") :
        if member not in list(DF_info["Phage"].unique()) :
            try : 
                family = df_family[df_family["Node_ID"] == member]["Family"].values[0]
                sub_fam = df_sub_fam[df_sub_fam["Node_ID"] == member]["Sub_family"].values[0]
                genus = df_genus[df_genus["Node_ID"] == member]["Genus"].values[0]
                a = {"family" : family, "sub_family" : sub_fam, "genus" : genus}
                taxonomy[row["VC"]] = a
                break
            except Exception as e :
                pass



4098it [01:33, 44.00it/s] 


In [31]:
df_family

{'VC_0_0': {'family': 'Inoviridae',
  'sub_family': 'Unclassified',
  'genus': 'Saetivirus'},
 'VC_1000_0': {'family': 'Unclassified',
  'sub_family': 'Gclasvirinae',
  'genus': 'Liefievirus'},
 'VC_1001_0': {'family': 'Unclassified',
  'sub_family': 'Gclasvirinae',
  'genus': 'Jolieduovirus'},
 'VC_1002_0': {'family': 'Vilmaviridae',
  'sub_family': 'Mclasvirinae',
  'genus': 'Unclassified'},
 'VC_1002_1': {'family': 'Vilmaviridae',
  'sub_family': 'Mclasvirinae',
  'genus': 'Bongovirus'},
 'VC_1002_2': {'family': 'Vilmaviridae',
  'sub_family': 'Mclasvirinae',
  'genus': 'Reyvirus'},
 'VC_1003_0': {'family': 'Vilmaviridae',
  'sub_family': 'Unclassified',
  'genus': 'Wildcatvirus'},
 'VC_1004_0': {'family': 'Unclassified',
  'sub_family': 'Unclassified',
  'genus': 'Pahexavirus'},
 'VC_1005_0': {'family': 'Zierdtviridae',
  'sub_family': 'Emilbogenvirinae',
  'genus': 'Gruunavirus'},
 'VC_1006_0': {'family': 'Zierdtviridae',
  'sub_family': 'Toshachvirinae',
  'genus': 'Ceetrepovirus

In [52]:
df_family

,Node_ID,Description,Family,Colour (Family)
0,MF417929,Uncultured Caudovirales phage clone 2F_1,Peduoviridae,#C5F88A
1,MH616963,crAssphage sp. isolate ctbg_1,Intestiviridae,#A293A2
2,MH552500,crAssphage sp. isolate ctcc615,Intestiviridae,#A293A2
3,BK010471,Carjivirus communis,Intestiviridae,#A293A2
4,GU339467,Mycobacterium phage RedRock,Unclassified,#D3D3D3
...,...,...,...,...
19715,AY526908,Bordetella phage BMP-1,Unclassified,#D3D3D3
19716,AJ270057,Chlamydia phage 2,Microviridae,#6B30B5
19717,AB016282,Bacillus phage phi105,Unclassified,#D3D3D3
19718,AF109874,Lactococcus phage Tuc2009,Siphoviridae,#CF83B6


In [38]:
dico_phylogeny = {}
for _, row in tqdm(df_vcontact.iterrows()) :
    for member in row["Members"].split(",") :
        if member in list(DF_info["Phage"].unique()) :
            try : 
                family = taxonomy[row["VC"]]["family"]
                sub_fam = taxonomy[row["VC"]]["sub_family"]
                genus = taxonomy[row["VC"]]["genus"]
            except KeyError :
                family , sub_fam, genus = ["Unclassified"] * 3
            phophage_id = df_representant[df_representant['prophage'] == f"{member}.fasta"]["prophage_id"].values[0]
            a = {"family" : family, "sub_family" : sub_fam, "genus" : genus, "VC" : row["VC"]}
            dico_phylogeny[phophage_id] = a 

4098it [03:23, 20.16it/s] 


In [39]:
dico_phylogeny

{'prophage_10812': {'family': 'Unclassified',
  'sub_family': 'Unclassified',
  'genus': 'Unclassified',
  'VC': 'VC_104_0'},
 'prophage_10938': {'family': 'Unclassified',
  'sub_family': 'Unclassified',
  'genus': 'Unclassified',
  'VC': 'VC_106_0'},
 'prophage_1879': {'family': 'Unclassified',
  'sub_family': 'Unclassified',
  'genus': 'Unclassified',
  'VC': 'VC_106_0'},
 'prophage_13402': {'family': 'Unclassified',
  'sub_family': 'Unclassified',
  'genus': 'Unclassified',
  'VC': 'VC_108_0'},
 'prophage_13613': {'family': 'Unclassified',
  'sub_family': 'Unclassified',
  'genus': 'Unclassified',
  'VC': 'VC_108_0'},
 'prophage_9745': {'family': 'Unclassified',
  'sub_family': 'Unclassified',
  'genus': 'Unclassified',
  'VC': 'VC_108_0'},
 'prophage_9701': {'family': 'Unclassified',
  'sub_family': 'Unclassified',
  'genus': 'Unclassified',
  'VC': 'VC_108_0'},
 'prophage_7721': {'family': 'Unclassified',
  'sub_family': 'Unclassified',
  'genus': 'Unclassified',
  'VC': 'VC_1097_

In [44]:
taxonomy["VC_1050_0"]

{'family': 'Unclassified',
 'sub_family': 'Unclassified',
 'genus': 'Trigintaduovirus'}